# Trading EDA

To install conda and binance packages to this notebook uncomment the code below

In [ ]:
#%conda install -c plotly plotly=5.9.0
#%conda install pip
#%conda install twisted
%pip install plotly==5.9.0
%pip install twisted
%pip install binance-connector==1.13.0
%pip install pandas
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached https://files.pythonhosted.org/packages/82/c3/f4be51831b0b40b44eca335cc0d8ddf0cec421d10a24f91116e03eb49b30/binance_connector-1.13.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d5/9f/9c0e3288b85f907a008f9d31318b0e4de31b2f67724a8745e633741f609c/pyOpenSSL-22.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/93/5a/5e93f280ec7be676b5a57f305350f439d31ced168bca04e6ffa64b575664/service_identity-21.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ca/91/6d9b8ccacd0412c08820f72cebaa4f0c0441b5cda699c90f618b6f8a1b42/requests-2.28.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c5/b5/c92d6640fd55cbbdd97c05800ab534d84197f7b485d89a9df981ab67cce3/autobahn-22.7.1.tar.gz
  Using cached https://files.pythonhosted.org/packages/6d/0c/5e67831007ba6cd7

ERROR: Command "/snap/jupyter/6/bin/python /snap/jupyter/6/lib/python3.7/site-packages/pip install --ignore-installed --no-user --prefix /tmp/pip-build-env-a04ftvty/overlay --no-warn-script-location --no-binary :none: --only-binary :none: -i https://pypi.org/simple -- 'setuptools>=40.6.0,!=60.9.0' wheel 'cffi>=1.12; platform_python_implementation != '"'"'PyPy'"'"'' 'setuptools-rust>=0.11.4'" failed with error code 1 in None
Note: you may need to restart the kernel to use updated packages.
  Using cached https://files.pythonhosted.org/packages/99/f0/f99700ef327e51d291efdf4a6de29e685c4d198cbf8531541fc84d169e0e/pandas-1.3.5.tar.gz
  Installing build dependencies ... done


Init binance client

In [ ]:
from binance.spot import Spot as Client
import os
import sys
import yaml

# Load 
def load_config():
    """
    Load config from cfg folder respecting the order: defaults, app.yaml, environment vars
    """
    # Defaults
    cfg_dir="./../biml/cfg"
    default_cfg_path = f"{cfg_dir}/app-defaults.yaml"
    with open(default_cfg_path, "r") as appdefaults:
        config = yaml.safe_load(appdefaults)

    # Custom config, should contain custom information,
    cfg_path = f"{cfg_dir}/app.yaml"
    if os.path.exists(cfg_path):
        with open(cfg_path) as app:
            config.update(yaml.safe_load(app))
    else:
        sys.exit(
            f"Config {cfg_path} not found. Please copy cfg/app-defaults.yaml to {cfg_path} "
            f"and update connection info there.")

    # Enviroment variabless
    config.update(os.environ)
    return config


# Create binance client
config=load_config()
key,secret,url=config["biml.connector.key"],config["biml.connector.secret"],config["biml.connector.url"]
client = Client(key=key, secret=secret, base_url=url, timeout=10)


Get account info

In [ ]:
account=client.account()
print(account)


Get orders from binance

In [ ]:
# Get orders
import pandas as pd
symbol="BTCUSDT"
orders_raw = client.get_orders(symbol)
orders=pd.DataFrame(orders_raw)
if not orders.empty: orders=orders.astype({'time': 'datetime64[ms]','updateTime':'datetime64[ms]'}).set_index('time',drop=False)
orders.tail()

In [ ]:
import numpy as np
botorders = orders.copy().astype({"time": "datetime64[ms]", "price": "float64", "origQty": "float64"})[["time","symbol","type","side","price","status","origQty"]].set_index("time", drop=False)
botorders["open_price"]=botorders["price"].shift(1)
botorders["qty"]=np.where(botorders["side"]=="SELL",botorders["origQty"]*-1,botorders["origQty"])
# botorders[(botorders["status"]=="FILLED")].tail()
botorders.loc[botorders["type"]=="STOP_LOSS_LIMIT","profit"]=(botorders["price"]-botorders["open_price"])*-1*botorders["qty"]

# Exclude initial closing orders
botorders = botorders[(botorders["status"]!="CANCELED") & (botorders["status"].shift(1)!="CANCELED")]
botorders = botorders[(botorders["type"]=="STOP_LOSS_LIMIT")|(botorders["type"].shift(-1)=="STOP_LOSS_LIMIT")]
botorders.tail()

Look at opened orders

In [ ]:
orders_opened=client.get_open_orders(symbol)
print(orders_opened)

Get candles from local history starting from earliest trade date ending by latest trade date.

In [ ]:
import os
import glob
import pandas as pd
from datetime import timedelta,date


def read_candles(start_date_inc, end_date_inc):
    """ Read days from start to end from local history"""
    symbol="BTCUSDT"    
    data_dir=f"./../data/{symbol}"
    print(f"start date:{start_date_inc}, end date:{end_date_inc}, all inclusive")
    candles = pd.DataFrame()
    cur_candles_list=[]
    for n in range(int ((end_date_inc - start_date_inc).days)+1):
        cur_date = start_date_inc + timedelta(n)    
        csv_path=f"{data_dir}/{cur_date}_{symbol}_1m.csv"     
        if not os.path.exists(csv_path): continue
        #print(f"Read {cur_date} candles from {csv_path}")
        cur_candles=pd.read_csv(csv_path,index_col="close_time")[["open","high","low","close","vol"]]
        cur_candles_list.append(cur_candles)
    candles = pd.concat(cur_candles_list)
    return candles

def clean_candles(df):
    """ Remove outliers """
    q_low = df["low"].quantile(0.01)
    q_high  = df["high"].quantile(0.99)
    df=df[df["high"]< q_high]
    df=df[df["low"] > q_low ]
    return df

# Read candles for last 3 days
last_date=orders.index.max().date()
first_date=last_date - timedelta(days=0)

candles = read_candles(first_date, last_date)
candles = clean_candles(candles)
candles.tail()


## EDA

### Look at trades quantity cumulative sum

## Look at trades on candlestick chart

In [ ]:
# Install a conda package in the current Jupyter kernel
#%conda install -c plotly plotly=5.9.0
# Draw candlechart
import plotly as py
from plotly import graph_objects as go
import pandas as pd

df=candles.copy()
#df=clean_candles(df)
fig = go.Figure(data=[ \
                    go.Candlestick(\
                        x=df.index,\
                        open=df['open'],\
                        high=df['high'],\
                        low=df['low'],\
                        close=df['close'])
                     ])

# Customize ay to distribute captions by y so they do not overlap by x
ay,aymax,aystep=0,50,10
# Annotate buy, sell on the chart

for traderec in botorders[(botorders["status"] == "FILLED") & (botorders.index>df.index.min())].iterrows():
    ay=(ay+aystep)%aymax
    trade=traderec[1]
    color='green' if trade['side']=="BUY" else 'red'
    text_short='buy' if trade['side']=="BUY"  else 'sell'
    status= f", profit: {str(trade['profit'])}" if ~np.isnan(trade['profit']) else ""
    #text=f"{trade['time']} {text_short} at {str(trade['price'])}, profit: {str(trade['profit'])}"
    text=f"{trade['time']} {text_short} at {str(trade['price'])} {status}"
    fig.add_annotation(x=trade['time'], \
                       y=trade['price'],\
                       text=text_short,\
                       hovertext=text,\
                       showarrow=True,arrowhead=5,arrowcolor=color,ay=-ay,arrowwidth=2,
                       font={'color':color}
                      )
fig.show()


## Profit

In [ ]:
#import matplotlib.pyplot as plt
import plotly.express as px
# Calc profit, considering only closing trades
profit=botorders[(botorders.status=="FILLED")&(botorders["profit"])].copy()
profit["cum_profit"] = profit["profit"].cumsum()

#Plot
fig = px.line(profit, x="time", y="profit")
fig.show()

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
# Calc profit, considering only closing trades
profit=botorders[(botorders.status=="FILLED")&(botorders["profit"])].copy()
#profit["profit"]=profit["price"].diff()*(profit["qty"]/profit["qty"])
profit["cum_profit"] = profit["profit"].cumsum()

#Plot
fig = px.line(profit, y="cum_profit")
fig.show()

## Look at the account

In [ ]:
print(account)